In [1]:
import requests
import csv
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

start_date = datetime(2023, 2, 1)
end_date = datetime(2023, 2, 2)

url = "https://news.daum.net/breakingnews/digital/device"

links = []
    # type을 set가 아닌 list로 초기화한 이유는 csv화일로 데이터를 저장할때 links에 데이터형을 dict로 하여
    # 컬럼제목을 연결시켜 주기 위해서 입니다. 

print(f"☞☞☞☞☞ url:   {url}")

while start_date <= end_date:
    date_str = start_date.strftime("%Y%m%d")
    print(f"\n\n====> 날짜:  {date_str}\n\n")
    
  
    num_page = 1
    last_page = False
    prev_article_links = None
    
    while not last_page:
        
        page_num_url = f"{url}?page={num_page}&regDate={date_str}"
        print(f"☞☞☞☞☞ page_num_url:   {page_num_url}")
        
        # User-Agent 정보 추가
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
        res = requests.get(page_num_url, headers=headers)

        # HTTP 요청이 성공했는지 확인
        if res.status_code == 200:
            print(f"☞☞☞☞☞ page_num_url 접속성공: {page_num_url}")
        else:
            print(f"☞☞☞☞☞ page_num_url 접속실패: {page_num_url}")
            break
            
        soup = BeautifulSoup(res.text, "html.parser")
            # soup = BeautifulSoup(res.content, "html.parser") 를 사용할 수도 있슴
            # res.content와 res.text의 차이는 반환되는 데이터 형식에 있습니다. 
            # 이들 간의 차이로 인해 BeautifulSoup에 전달되는 인자도 약간 다릅니다.
            # res.content: content 속성은 응답의 본문을 바이트(byte) 형태로 반환합니다. 
            # 이는 이미지, 동영상, PDF 등 바이너리 형식의 데이터를 처리할 때 유용합니다. 
            # HTML 문서의 경우 인코딩이 다를 수 있으므로, BeautifulSoup에서는 해당 
            # 인코딩을 자동으로 감지하고 적용합니다.
            # res.text: text 속성은 응답의 본문을 유니코드 문자열로 반환합니다. 
            # 일반적으로 텍스트 기반 문서를 처리할 때 사용됩니다. 
            # requests 라이브러리는 기본적으로 인코딩을 자동으로 감지하여 적용하므로, 
            # 이 경우에도 대부분의 상황에서 적절한 인코딩이 적용됩니다.
            # 대부분의 경우, 두 가지 방식 모두 HTML 문서를 처리하는데 문제가 없습니다. 
            # 하지만 일부 상황에서는 인코딩이 잘못 감지되거나 처리되는 경우가 있을 수 있으므로, 
            # 이러한 경우에는 명시적으로 인코딩을 지정해주는 것이 좋습니다.
            # 결론적으로, soup = BeautifulSoup(res.content, "html.parser")와 
            # soup = BeautifulSoup(res.text, "html.parser")는 동일한 결과를 가져오는 경우가 대부분이지만, 
            # 일부 상황에서 인코딩 문제로 차이가 발생할 수 있습니다. 
            # 기본적으로는 res.text를 사용하여 HTML 문서를 처리하는 것이 좋습니다.
            # 웹 크롤링 및 스크레이핑 작업에서 대부분의 경우 HTML, XML 같은 텍스트 기반 문서를 처리하게 되므로,
            # 대부분의 상황에서 res.text를 사용하는 것이 일반적입니다.
            # res.content는 바이너리 데이터를 처리해야 하는 경우에 주로 사용됩니다. 
            # 이미지, 동영상, PDF 파일 등의 바이너리 형식의 데이터를 처리할 때 res.content를 사용하면 좋습니다.
         
        # robots.txt 파일 참조
        robot_check = soup.find('meta', attrs={'name': 'robots'})
            # robots 메타 태그는 웹 페이지에 대한 검색 엔진 봇의 인덱싱과 크롤링 행동을 제어하는 방법 중 하나입니다.
            # 위 코드는 BeautifulSoup을 사용하여 HTML에서 'meta' 태그 중 'name' 속성이 'robots'인 것을 찾습니다. 
            # 해당 태그가 존재하면 robot_check 변수에 저장하는 코드입니다.        
        if robot_check and 'nofollow' in robot_check['content']:
            print('로봇체크결과 해당페이지 {page_num_url} 가 크롤링을 제한하고 있습니다.')
            break
            
            # 위 코드는  robot_check 변수에 저장된 메타 태그가 존재하고, 
            # 해당 태그의 'content' 속성에 'nofollow'라는 값이 포함되어 있는지 확인합니다. 
            # 'nofollow'는 검색 엔진 봇에게 이 웹 페이지의 링크를 따라가지 않도록 지시하는 것입니다. 
            # 즉, 해당 웹 페이지가 크롤링에 대한 제한을 설정하고 있음을 나타냅니다.
            # 이 경우 코드는 '~~~크롤링을 제한하고 있습니다.' 라는 메시지를 출력하고, 
            # break 문으로 해당 웹 페이지 크롤링을 중단하도록 하는 코딩입니다.

        # 기사의 링크를 추출하여 리스트에 추가
        article_links = soup.select(".list_news2 a.link_txt")
        print(f"☞☞☞☞☞ article_links : {article_links}\n\n\n" )
        
        if not article_links:
                # article_links 리스트가 비어있는지 확인합니다. 만약 리스트가 비어있다면, 
                # CSS 선택자를 사용하여 기사 링크를 찾는 것에 실패한 것으로 간주하고 break처리합니다.
            
            print(f"\n☞ CSS선택자 .list_news2 a.link_txt 에 해당하는 링크를 찾을 수 없습니다.")
            
            break
        
        
        if prev_article_links and prev_article_links == article_links:
                # 위 조건문은 True and True 일때 
                # last_page 에 True를 할당하는 코딩입니다.
                # 즉, prev_article_links(이전 페이지의 기사 링크)가 비어있지 않고, 
                # 이전 페이지의 기사 링크와 현재 페이지의 기사 링크(article_links)가 동일한 경우를 확인합니다. 
                # 이 경우, 현재 페이지는 마지막 페이지로 간주하는 코드를 아래에 씁니다.
            last_page = True
            print(f"\n☞☞☞☞☞ {date_str}에서 {num_page - 1}가 마지막 페이지입니다.")
            break
        
        for link in article_links:
    
            article_url = link["href"]
                    # article_links 리스트에서 가져온 기사 링크의 주소를 article_url 변수에 저장하는 부분이다.
                    # link는 article_links 리스트에서 하나씩 기사 링크를 가리키는 객체이며, 
                    # "href"는 해당 객체에서 href 속성을 의미합니다. 즉, article_url 변수에는 현재 가리키는 기사 링크의 주소가 저장된다.
                    # href 속성은 HTML 태그 중 <a> 태그에 사용되는 속성으로, 
                    # 해당 링크의 주소를 지정합니다. href 속성값은 일반적으로 URL 주소를 가리키며, 
                    # 클릭 시 해당 주소로 이동하게 됩니다. 예를 들어, <a href="http://www.example.com">Click here</a>와 같이 
                    # 작성하면 "Click here"라는 텍스트가 링크로 표시되며, 클릭 시 http://www.example.com 으로 이동하게 됩니다.
                
                
            article_res = requests.get(article_url)
                # article_res 변수에 해당 기사 링크로 GET 요청을 보내서 응답을 받아옵니다.
                
                # HTTP 요청이 성공했는지? 여부를 확인하기 위한 코드
            if article_res.status_code == 200:
                print(f"☞☞☞☞☞ 접속성공 article_url : {article_url}")
                print(f"☞☞☞☞☞ {article_url} 의 응답코드 : {article_res}")
                
                    # 추출: html전체 
                article_soup = BeautifulSoup(article_res.text, "html.parser")
                    # 추출: 기사제목
                title = article_soup.select_one(".tit_view").get_text()
                    # 추출: 일자시각
                date = article_soup.select_one(".info_view .num_date").get_text()
                    # 추출: 기사내용
                contents = article_soup.select(".article_view p")
                content = "\n".join([c.get_text().strip() for c in contents])
                
                links.append({'기사링크': article_url, '기사제목': title, '일자시각': date, '기사내용': content})
                    # links에 데이터를 추가할때 dict형으로 했습니다. 이유는 csv화일을 만들때 컬럼의 제목을 넣어주기 위해서입니다.
                
                print(f"☞☞☞☞☞ 기사링크: {article_url}")
                print(f"☞☞☞☞☞ 기사제목: {title}")
                print(f"☞☞☞☞☞ 일자시각: {date}")
                print(f"☞☞☞☞☞ 페이지번호: {num_page}")
                print(f"☞☞☞☞☞ 기사내용 \n{content}\n\n")
                     
            else:
                print(f"☞☞☞☞☞ 접속실패 article_url : {article_url}")
                print(f"☞☞☞☞☞ {article_url} 의 응답코드 : {article_res}")
                break                
            
            # 마지막 페이지가 아니면 다음 페이지로 이동
            # 이때 article_links의 정보를 prev_article_links에 할당하여 넘겨준다.
        if not last_page:
            prev_article_links = article_links
            num_page += 1
            
        # 다음 날짜로 이동
    start_date += timedelta(days=1)
    
    with open('NewsCrawling_2.csv', 'w', encoding='utf-8', newline='') as file:
            # 'NewsCrawling_2.csv'라는 이름의 CSV 파일을 쓰기 모드('w')로 열고, 인코딩을 'utf-8'로 설정합니다. 
            # newline=''은 줄바꿈 문자를 플랫폼에 맞게 처리하기 위한 설정입니다. 
            # with문은 파일을 열고 작업을 마친 후 자동으로 파일을 닫는 기능을 제공합니다.
            
        fieldnames = ['기사링크', '기사제목', '일자시각', '기사내용']
            # CSV 파일의 열 이름(헤더)을 설정합니다. 여기서는 기사 링크, 기사 제목, 
            # 기사 작성 시각, 기사 내용이 각각의 헤더로 설정됩니다.
            # 위에서 코딩한 links.append({'기사링크': article_url, '기사제목': title, '일자시각': date, '기사내용': content})
            # links의 key와 열 이름(헤더)가 불일치할 경우 에러메시지가 발생됩니다.
        writer = csv.DictWriter(file, fieldnames=fieldnames)
            # csv 모듈의 DictWriter 클래스를 사용하여 CSV 파일에 딕셔너리 형태의 
            # 데이터를 쓸 수 있는 객체를 생성합니다. 위에서 정의한 fieldnames를 이용해 열 이름을 지정합니다.
        writer.writeheader()
            # 위의 코딩은 CSV 파일에 헤더(열 이름)를 쓰는 작업을 수행합니다.
        for link in links:
            # links라는 리스트에 저장된 각각의 기사 정보(딕셔너리 형태)에 대해 반복문을 실행합니다.
            writer.writerow(link)
            # links 리스트의 각 기사 정보(link)를 순서대로 CSV 파일에 씁니다. 
            # 딕셔너리 형태의 데이터가 CSV 파일의 각 열에 맞춰 저장됩니다.
            
    



☞☞☞☞☞ url:   https://news.daum.net/breakingnews/digital/device


====> 날짜:  20230201


☞☞☞☞☞ page_num_url:   https://news.daum.net/breakingnews/digital/device?page=1&regDate=20230201
☞☞☞☞☞ page_num_url 접속성공: https://news.daum.net/breakingnews/digital/device?page=1&regDate=20230201
☞☞☞☞☞ article_links : [<a class="link_txt" href="https://v.daum.net/v/20230201190606054">벤츠 “미래 비전 담은 12종 신차 내놓을 것”…첫 포문 ‘더 뉴 EQS SUV’ 출시</a>, <a class="link_txt" href="https://v.daum.net/v/20230201154217912">새 레이아웃·기능 갖춘 안드로이드 오토 '쿨워크', 직접 써보니</a>, <a class="link_txt" href="https://v.daum.net/v/20230201151503830">[스케일업] 엘핀 [3] 학교·기업·근로자 출결관리의 모든 것 ‘아임히어·아임히어워크’</a>, <a class="link_txt" href="https://v.daum.net/v/20230201144247413">엔엑스플러스, 신작 무협 방치RPG '요도봉인기' 출시</a>, <a class="link_txt" href="https://v.daum.net/v/20230201143449017">대동, 로봇사업 진출… KIRO와 개발 협력</a>, <a class="link_txt" href="https://v.daum.net/v/20230201142832807">삼성 갤럭시S23, 실제로 보면 이런 모습?</a>, <a class="link_txt" href="https://v.daum.net/v/2023020

☞☞☞☞☞ 접속성공 article_url : https://v.daum.net/v/20230201151503830
☞☞☞☞☞ https://v.daum.net/v/20230201151503830 의 응답코드 : <Response [200]>
☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230201151503830
☞☞☞☞☞ 기사제목: [스케일업] 엘핀 [3] 학교·기업·근로자 출결관리의 모든 것 ‘아임히어·아임히어워크’
☞☞☞☞☞ 일자시각: 2023. 2. 1. 15:15
☞☞☞☞☞ 페이지번호: 1
☞☞☞☞☞ 기사내용 
[스케일업 x SBA] 스케일업팀이 서울산업진흥원(SBA)과 함께 ‘2022년 하반기 스케일업 프로그램’을 진행합니다. 스케일업팀은 이번 프로그램에 참여하는 각각의 스타트업이 현재 진행하고 있는 사업 전반에 대해 소개하고, 다음 단계로 나아가기 위해 도전하고 있는 문제를 해결할 수 있도록 각자의 전문 영역에서 활동하는 여러 전문가를 연결해 도움을 드리고자 합니다.
[IT동아 차주경 기자] 두 차례에 걸친 스케일업 프로그램에서 엘핀만의 기술인 '복합 위치인증'의 원리와 장점, 활용 영역을 소개했다. 기존 위치 인증에 '이동통신사 기지국'의 정보를 더해 위변조 불가능한, 손쉽고 간편하게 활용 가능한 새로운 인증 기술이다. 이어 이 기술을 더욱 다양한 분야에 전파해 가치를 만들 전략을 살펴봤다. 전략과 함께 엘핀이 풀어야 할 도전 과제, 강화할 기본기와 바라볼 시장도 가늠했다.
스케일업코리아가 초빙한 전문가들은 엘핀에게 다양한 조언을 건넸다. 골자는 복합 위치인증만이 주는 장점을 명확히 소개하고 다양한 활용 사례를 마련할 것, 이 활용 사례들을 보안과 인증 편의 확보를 원하는 국내외 기업에 적극 공급할 것, 이를 위해 투자금을 유치해 체력을 기를 것이다.

복합 위치인증은 쓰기 쉽고 편리하다. 위변조 불가능하기에 기본의 보안 장벽에 또 하나의 두터운 장벽을 덧대는 효과를 낸다. 인증 수단으로 위치를 사용하기에 다루기 쉽고 적용 영역

☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230201132104420
☞☞☞☞☞ 기사제목: [아트테크A-Z] 자산 포트폴리오 다각화를 위해 미술품 투자에 눈 떠야 할 때
☞☞☞☞☞ 일자시각: 2023. 2. 1. 13:21
☞☞☞☞☞ 페이지번호: 1
☞☞☞☞☞ 기사내용 
[IT동아]
[편집자주] 날로 관심이 뜨거워지고 있는 아트테크(미술품 투자)에 입문하려는 이들을 위해 아트 컨설팅 기업 테사 에셋과 함께 아트테크란 무엇이고, 어떻게 하는 것인지 차근차근 짚어주는 연재를 마련했습니다. 본문의 내용 또는 의견은 본지의 편집 방향과 다를 수 있습니다.
최근 문화체육관광부의 발표에 따르면 2022년 국내 미술 시장 규모는 역대 최대 수치인 1조 원을 돌파했다. 지난해 글로벌 메가 갤러리와 아트페어가 국내에 진출하며 유례없는 미술품 유통액을 달성했고 이는 국내 미술시장의 확대로 이어지며 새로운 전환점을 맞고 있다.
시장 확대와 맞물려 미술품 수집에 대한 관심도 높아지고 있다. 미술품 수집은 취향을 반영한 작품을 구매해 감상하려는 심미적 목적도 있겠으나, 최근엔 미술품을 활용한 조각투자가 큰 인기를 끌면서 투자 수요도 크게 늘고 있다.
미술품 투자의 높은 인기는 미술품이 유형자산으로서, 비교적 안전한 대체 투자 상품이기 때문이다. 실제로 씨티은행이 발표한 자료에 따르면 2020년 팬데믹 동안 미국 부동산, 헤지 펀드와 같은 타 자산군이 각각 -14.5%, 2%의 수익률을 기록한 반면, 현대 미술 작품은 6.7%의 수익률을 보였다. 고인플레이션 시기에 현대 미술 작품이 S&P 500 지수를 약 2.5배 앞서 나가며 타 자산군을 능가했음을 시사한다.
또한 경제시장의 위축과 혼란 속에서도 미술시장은 빠른 회복력과 건재함을 보여줬다. 팬데믹이었던 2020년 대비 2021년 미술 시장 거래규모는 29% 증가한 651억달러(약 79조원)를 기록하며, 1년 만에 2019년 수준 이상으로 빠른 회복세를 보였다. 2022년에는 글로벌 3대 경매사(소더비, 크리스티

☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230201100755764
☞☞☞☞☞ 기사제목: 젠하이저, 'ISE 2023'서 최신 오디오 솔루션 공개
☞☞☞☞☞ 일자시각: 2023. 2. 1. 10:07
☞☞☞☞☞ 페이지번호: 1
☞☞☞☞☞ 기사내용 
(지디넷코리아=신영빈 기자)독일 오디오 기업 젠하이저가 3일까지(현지시간) 스페인 바르셀로나에서 열리는 유럽 최대 상업용 디스플레이 전시회 ‘ISE 2023’에 참가해 최신 오디오 솔루션을 선보인다고 1일 밝혔다.
젠하이저는 정교한 무선 기술을 기반으로 안정적인 연결성과 확장성을 제공하는 디지털 무선 오디오 시스템 ‘EW-DX’를 공개한다.

‘EW-DX’는 바디팩, 핸드 마이크, 테이블-스탠드 타입 등의 송신기와 수신기 등으로 구성됐다. 자동으로 주파수를 스캔하고 많은 채널을 할당할 수 있어 기업, 교육 기관 등에서 손쉽게 사용할 수 있다.
젠하이저는 실제 회의 환경을 구현해 최신 오디오 기술을 체험할 수 있는 데모 공간을 마련한다.
젠하이저는 화상회의에서 사용하는 천장형 마이크인 ‘TCC2’로 화자의 음성에 초점을 맞춰 전파를 집중시키는 ‘다이나믹 빔포밍’ 기술과 음성의 뚜렷함을 향상시키는 기술인 ‘트루 보이스리프트’, 원치 않는 배경 소음을 제거하기 위한 구역 제어 기능 등을 소개한다.

이 외에도 젠하이저는 ▲스피치·강의에 최적화된 마이크 시스템 ‘SL DW’ ▲음성을 텍스트로 기록하는 소규모 회의실 전용 지능형 스피커 ‘TC ISP’ ▲브라우저 기반 원격 관리 소프트웨어 도구 ‘컨트롤 콕핏’ ▲스마트폰 기반의 보조 청취 솔루션 ‘모바일 커넥트’ 등을 선보인다.
젠하이저는 이번 전시회에서 다양한 고객의 요구 사항을 충족하기 위해 보스, 캐논, 크레스톤, 크래머, Q-SYS, 실리카 등 글로벌 파트너와 제휴를 맺고 파트너사 부스에서 오디오 솔루션을 선보인다.
젠하이저 관계자는 “젠하이저는 회의나 컨퍼런스콜, 강의 등 다양한 유형의 커뮤니케이션 환경에서 최적의 오디오 솔루션을 제공하기 위해

☞☞☞☞☞ page_num_url 접속성공: https://news.daum.net/breakingnews/digital/device?page=3&regDate=20230201
☞☞☞☞☞ article_links : [<a class="link_txt" href="https://v.daum.net/v/20230201093505279">로지텍, e스포츠 게임단 '브리온'과 파트너십 연장</a>, <a class="link_txt" href="https://v.daum.net/v/20230201091142352">우아한형제들 서빙로봇 자회사 ‘비로보틱스’ 출범</a>, <a class="link_txt" href="https://v.daum.net/v/20230201090019902">삼성전자 '데이코', 미국주방욕실협회서 '혁신 쇼룸상' 수상</a>, <a class="link_txt" href="https://v.daum.net/v/20230201082906990">애플, 맥북도 접나…"2025년 20.5인치 폴더블 노트북 출시"</a>]




☞☞☞☞☞ 20230201에서 2가 마지막 페이지입니다.


====> 날짜:  20230202


☞☞☞☞☞ page_num_url:   https://news.daum.net/breakingnews/digital/device?page=1&regDate=20230202
☞☞☞☞☞ page_num_url 접속성공: https://news.daum.net/breakingnews/digital/device?page=1&regDate=20230202
☞☞☞☞☞ article_links : [<a class="link_txt" href="https://v.daum.net/v/20230202201201886">언팩 행사 '명품 조연' 된 갤럭시 북…뛰어난 가성비 호평</a>, <a class="link_txt" href="https://v.daum.net/v/20230202185402582">난방비 폭탄…감면 대상이라면 ‘에너

...

☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202105652284
☞☞☞☞☞ 기사제목: 에이수스, 냉각 강화한 게임용 노트북 3종 출시
☞☞☞☞☞ 일자시각: 2023. 2. 2. 10:56
☞☞☞☞☞ 페이지번호: 2
☞☞☞☞☞ 기사내용 
(지디넷코리아=권봉석 기자)에이수스코리아가 CES 2023에서 공개한 고성능 게임용 노트북 3종을 출시했다.
ROG 스트릭스 스카 18·16은 인텔 13세대 코어 i9-13980HX 프로세서와 엔비디아 지포스 RTX 4090 그래픽칩셋 등을 탑재한 제품이다. 스카 18은 18인치, 240Hz 고주사율 디스플레이를 탑재해 게임 몰입감을 높였다.

세개의 보조 팬으로 최적의 공기 흐름을 구현한 '트라이팬' 기술, 프로세서 온도를 기존 서멀그리스 적용시 대비 최대 15도 더 낮출 수 있는 액체금속 등을 적용해 냉각 성능을 강화했다.
ROG 제피러스 M16은 두께 19.9mm, 무게 2.1kg 본체로 동급 제품 대비 휴대성을 강화했다. 인텔 13세대 코어 i9-13900H 프로세서, 엔비디아 지포스 RTX 4090 그래픽칩셋을 탑재했다.
세 제품의 출고가는 모두 400만원 대 중반부터 시작한다. 옥션·G마켓, 11번가와 쿠팡 등에서 제품 구매시 할인 쿠폰과 전용 백팩 등을 추가 제공한다.
권봉석 기자(bskwon@zdnet.co.kr)


☞☞☞☞☞ 접속성공 article_url : https://v.daum.net/v/20230202103547251
☞☞☞☞☞ https://v.daum.net/v/20230202103547251 의 응답코드 : <Response [200]>
☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202103547251
☞☞☞☞☞ 기사제목: AMD, 라이젠 7000 X3D 프로세서 오는 28일 출시
☞☞☞☞☞ 일자시각: 2023. 2. 2. 10:35
☞☞☞☞☞ 페이지번호: 2
☞☞☞☞☞ 기사내용 
(지디넷코리아=권봉석 기자)AMD가 데스크톱PC용 라이젠 7000

☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202093802526
☞☞☞☞☞ 기사제목: 캐논코리아, 3월 말까지 카메라 정품등록 행사 진행
☞☞☞☞☞ 일자시각: 2023. 2. 2. 09:38
☞☞☞☞☞ 페이지번호: 2
☞☞☞☞☞ 기사내용 
(지디넷코리아=권봉석 기자)캐논코리아가 오는 3월 말까지 카메라 구매자 대상 정품등록 행사를 진행한다.
대상 제품은 EOS R5·R6 마크Ⅱ·R·RP·R10 등 미러리스 카메라 6종, EOS 5D 마크Ⅳ·6D 마크Ⅱ·90D·850D 등 DSLR 카메라 4종이다.

오는 3월 말까지 제품을 구매하고 4월 7일까지 정품등록을 마치면 선택한 사은품이 배송된다.
제품에 따라 카메라 가방과 에코이너케이스, 포토프린터, 메모리카드, 정품 배터리, 모바일 상품권 등을 추가 지급한다. 일부 사은품은 선착순 한정 수량으로 제공된다.
행사 개요와 대상 제품 제원 등 상세 정보는 캐논코리아 웹사이트에서 확인할 수 있다.
권봉석 기자(bskwon@zdnet.co.kr)


☞☞☞☞☞ 접속성공 article_url : https://v.daum.net/v/20230202093038256
☞☞☞☞☞ https://v.daum.net/v/20230202093038256 의 응답코드 : <Response [200]>
☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202093038256
☞☞☞☞☞ 기사제목: 넷기어, 이달 말까지 뮤럴 디지털 액자 할인판매
☞☞☞☞☞ 일자시각: 2023. 2. 2. 09:30
☞☞☞☞☞ 페이지번호: 2
☞☞☞☞☞ 기사내용 
(지디넷코리아=권봉석 기자)넷기어코리아가 이달 말까지 뮤럴 디지털 액자를 할인판매한다. 행사 대상 제품은 21.5인치(MC321), 27인치(MC327) 등 2종류다.
21.5인치 제품은 25% 할인판매하며 액자 방향을 가로·세로로 전환할 수 있는 회전 마운트가 제공된다.

27인치 제품은 15% 할인하며 15.6인치 제품을 추가 제공한다.
두 제품 모두 

☞☞☞☞☞ page_num_url 접속성공: https://news.daum.net/breakingnews/digital/device?page=3&regDate=20230202
☞☞☞☞☞ article_links : [<a class="link_txt" href="https://v.daum.net/v/20230202055715688">베일 벗은 삼성 갤럭시 'S23 시리즈'…2억 화소 카메라 장착</a>, <a class="link_txt" href="https://v.daum.net/v/20230202053850548">삼성전자, 퀄컴·구글과 XR 개발 나선다...'언팩'서 깜짝 발표</a>, <a class="link_txt" href="https://v.daum.net/v/20230202052405438">갤럭시S23, 가격 15만원 인상…해외는 동결</a>, <a class="link_txt" href="https://v.daum.net/v/20230202041609797">외신 "갤럭시S23 2억 화소 카메라 센서 놀랍다"</a>, <a class="link_txt" href="https://v.daum.net/v/20230202030152276">삼성, 프리미엄 노트북 시장 정조준…갤럭시북 라인업 확대</a>, <a class="link_txt" href="https://v.daum.net/v/20230202030139259">친환경 입은 갤럭시S23, 재활용 소재 부품 2배 늘렸다</a>, <a class="link_txt" href="https://v.daum.net/v/20230202030122243">[영상] S시리즈 13년 역사가 '반짝'...MZ 취향 저격한 갤S23 체험관</a>, <a class="link_txt" href="https://v.daum.net/v/20230202030100206">삼성전자, 2억 화소 괴물 카메라 '갤럭시S23' 전격 공개</a>, <a class="link_txt" href="https://v.daum

☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202030139259
☞☞☞☞☞ 기사제목: 친환경 입은 갤럭시S23, 재활용 소재 부품 2배 늘렸다
☞☞☞☞☞ 일자시각: 2023. 2. 2. 03:01
☞☞☞☞☞ 페이지번호: 3
☞☞☞☞☞ 기사내용 
(지디넷코리아=류은주 기자)[샌프란시스코(미국)=류은주 기자] 삼성전자가 환경·사회·지배구조(ESG) 경영에 발맞춰 갤럭시 스마트폰에 친환경 제품을 적극 활용한다.
삼성전자는 1일(현지시간) 파트너사와 개방적 협력을 통해 갤럭시S23 시리즈에 재활용 소재 부품 사용을 전작 보다 2배 늘렸다고 밝혔다.

갤럭시S23 울트라 모델에는 총 12개의 재활용 소재 적용 부품이 사용됐고, 갤럭시S23+와 S23은 각각 11개 재활용 소재가 적용됐다. 지난해 출시한 갤럭시S22의 경우 폐어망을 재활용한 플라스틱 소재를 포함해 총 6개의 내장 부품에 재활용 플라스틱이 적용된 바 있다.
특히 갤럭시 플래그십 스마트폰의 외장 부품에도 재활용 소재가 적용된 것은 갤럭시S23 시리즈가 처음이다.
갤럭시S23 울트라는 전작에서 사용했던 폐어망∙폐생수통의 재활용 플라스틱뿐만 아니라 ▲폐페트(PET)병을 재활용한 플라스틱 ▲공정 중 발생하는 부산물을 재활용한 알루미늄 ▲공정 중 발생하는 파유리를 재활용한 글라스를 새롭게 개발해 추가 적용했다.
폐어망 재활용 소재를 20% 사용해 만든 재활용 플라스틱(PA)은 내부 S펜 커버, 하단 스피커 모듈 등에 적용됐다. 이를 통해 삼성전자는 올 한해 동안 약 15톤 이상의 폐어망을 수거하여 재활용하는데 기여할 수 있을 것으로 기대하고 있다.
폐생수통 재활용 소재를 20% 사용해 만든 재활용 플라스틱(PC)은 상단∙하단 스피커 모듈, 사이드키, 볼륨키에 적용됐다. 폐페트병 재활용 소재를 10% 사용해 만든 재활용 플라스틱(PBT)은 외장 케이스 프론트에 사용됐다. 또한 동일 소재를 80% 사용해 만든 필름은 제품 후면의 글라스 내부에 사용됐다.

공정 중 발생하는 알루미늄 부산물을 

☞☞☞☞☞ 접속성공 article_url : https://v.daum.net/v/20230202030018160
☞☞☞☞☞ https://v.daum.net/v/20230202030018160 의 응답코드 : <Response [200]>
☞☞☞☞☞ 기사링크: https://v.daum.net/v/20230202030018160
☞☞☞☞☞ 기사제목: 베일벗은 갤럭시S23, 2억 화소 카메라·최강 AP로 승부수
☞☞☞☞☞ 일자시각: 2023. 2. 2. 03:00
☞☞☞☞☞ 페이지번호: 3
☞☞☞☞☞ 기사내용 
(지디넷코리아=류은주 기자)[샌프란시스코(미국)=류은주 기자] 지난해 게임 옵티마이징 논란(GOS) 논란으로 곤욕을 치른 삼성전자가 역대급 카메라 성능과 최신 애플리케이션 프로세서(AP)를 장착한 플래그십 스마트폰 갤럭시S23를 앞세워 자존심 회복에 나섰다.
삼성전자는 1일(현지시간) 미국 샌프란시스코 머소닉 오디토리움에서 ‘갤럭시 언팩 2023(Galaxy Unpacked 2023: Share the Epic)'을 열고 '갤럭시S23 시리즈'를 전격 공개했다. 3년 만에 오프라인으로 열린 이번 언팩 행사에는 전 세계 미디어·파트너 등 2천여명 참석했다.

노태문 MX사업부장 사장은 이날 "프리미엄 스마트폰 성능의 기준을 재정의하고 성능과 품질면에서 모두 역대 갤럭시S 시리즈 중 최고라는 확신을 드릴 수 있는 제품"이라며 자신감을 나타냈다.
앞서 1월 노 사장은 삼성전자 뉴스룸에 쓴 기고문에서도 "새로운 갤럭시S 시리즈 울트라 모델은 성능과 품질 면에서 최고 중의 최고라는 확신을 드릴 제품"이라며 "갤럭시노트 경험을 울트라에 집약해 파워와 성능 모두에 최고의 혁신을 담았다"고 강조한 바 있다.
■ 세계 첫 2억 화소 카메라 성능 방점…갤럭시 전용 퀄컴 AP 탑재
갤럭시S23 울트라는 스마트폰 사상 역대 최고인 2억 화소 카메라를 탑재해 카메라 성능을 강화했다. 손 떨림 보정을 위한 OIS 각도도 2배 높였다.
갤럭시S23+와 갤럭시S23은 5천만 화소 